In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df_raw = pd.read_csv('std_data/MCTest/mc160/mc160.train.csv',index_col=0)
train_df_raw.head()

,id,author,work_times,article,question_type,question,A,B,C,D,answer
0,mc160.train.0,1844466225,2383,Tom had to fix some things around the house. H...,multiple,What was the hardest thing for Tom and his fri...,Door,House,Window,Toilet,C
1,mc160.train.0,1844466225,2383,Tom had to fix some things around the house. H...,one,What did Tom need to fix first?,Door,House,Window,Toilet,D
2,mc160.train.0,1844466225,2383,Tom had to fix some things around the house. H...,multiple,Jim didn't bring which person with him?,Jim,Dolly,Molly,Holly,B
3,mc160.train.0,1844466225,2383,Tom had to fix some things around the house. H...,one,Who was Tom's best friend?,Molly,Holly,Jim,Dolly,C
4,mc160.train.1,88979267,12668,Lisa has a pet cat named Whiskers. Whiskers is...,multiple,What day is Whisker's Birthday?,Today,Last year,Friday,Saturday,D


In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
#gold = le.fit_transform(df_train['answer'])

In [6]:
import torch
from transformers import BertConfig, BertTokenizer,CONFIG_NAME, WEIGHTS_NAME,BertForMultipleChoice
from torch.nn.modules import Softmax

pretrained_model = "bert-base-uncased"

config = BertConfig.from_pretrained(
            pretrained_model,
            num_labels=4
)
tokenizer = BertTokenizer.from_pretrained(pretrained_model)
model = BertForMultipleChoice.from_pretrained(pretrained_model,config=config)

# high-level 顯示此模型裡的 modules
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=1, bias=True)


In [110]:
#model.config

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
#_, acc = get_predictions(model, trainloader, compute_acc=True)
#print("classification acc:", acc)

device: cpu


In [62]:
#df_train = df_train.sample(frac=0.01, random_state=9527)

In [63]:
#df_train[['article','question','answer']]

In [166]:
predrictions = []
for i in range(10):
    try:
        fact_sent = train_df['article'][i]
        answers = [train_df[opt][i] for opt in ['A','B','C','D']]
        labels = torch.tensor(0).unsqueeze(0)
        fact_sent = [fact_sent for _ in range(len(answers)) ]
        
        encoding = tokenizer(list(zip(fact_sent, answers)), return_tensors='pt', padding=True)
        outputs = model(**{k: v.unsqueeze(0) for k,v in encoding.items()}, labels=labels)
        loss, logits = outputs[:2]

        predrictions.append(torch.max(logits.data.indices.tolist()[0], 1))

    except:
        predrictions.append([])

In [193]:
tokens = tokenizer.tokenize('[SEP][CLS]')
tokenizer.convert_tokens_to_ids(tokens)

[102, 101]

# Prepare Training set

In [29]:
train_df_std = pd.read_csv('std_data/MCTest/mc160/mc160.train.csv',index_col=0)
train_df = pd.DataFrame({})
train_df["labels"] = le.fit_transform(train_df_std['answer'])

for i in ["A","B","C","D"]:
    train_df[f"input_ids_{i}"] = train_df_std['article']+ '[SEP]' +train_df_std['question']++ '[SEP]' +train_df_std['i']

In [32]:
le.transform(le.classes_)

array([0, 1, 2, 3])

In [33]:
train_df.head()

,labels,article_question,A,B,C,D
0,2,[CLS]Tom had to fix some things around the hou...,Door,House,Window,Toilet
1,3,[CLS]Tom had to fix some things around the hou...,Door,House,Window,Toilet
2,1,[CLS]Tom had to fix some things around the hou...,Jim,Dolly,Molly,Holly
3,2,[CLS]Tom had to fix some things around the hou...,Molly,Holly,Jim,Dolly
4,3,[CLS]Lisa has a pet cat named Whiskers. Whiske...,Today,Last year,Friday,Saturday


In [34]:
from sklearn.model_selection import train_test_split
# create train and validation set 
train, val = train_test_split(train_df, test_size=0.1)

In [35]:
train.to_csv("train.csv")
val.to_csv("val.csv")
val.to_csv("test.csv")

# Starts Traning 

In [19]:
def save_checkpoint(save_path, model, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):

    if save_path == None:
        return
    
    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_metrics(load_path):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']



In [12]:
# Preliminaries
import torchtext
from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

# Models
import torch.nn as nn
import torch.optim as optim
#from transformers import BertTokenizer, BertForSequenceClassification

In [13]:
# Model parameter
MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

# Fields
label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)
text_field = Field(use_vocab=False, tokenize=tokenizer.encode, lower=False, include_lengths=False, batch_first=True,
                   fix_length=MAX_SEQ_LEN, pad_token=PAD_INDEX, unk_token=UNK_INDEX)
fields = [('label', label_field), ('article_question', text_field), ('A', text_field),('B', text_field),('C', text_field),('D', text_field)]

/Users/gavin/anaconda3/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [14]:
# TabularDataset
train, valid, test = TabularDataset.splits(path="./", train='train.csv', validation='val.csv',
                                           test='test.csv', format='CSV', fields=fields, skip_header=True)

/Users/gavin/anaconda3/lib/python3.7/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/Users/gavin/anaconda3/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
Token indices sequence length is longer than the specified maximum sequence length for this model (530 > 512). Running this sequence through the model will result in indexing errors


In [15]:
train_iter = BucketIterator(train, batch_size=16, sort_key=lambda x: len(x.article_question),
                            device=device, train=True, sort=True, sort_within_batch=True)
valid_iter = BucketIterator(valid, batch_size=16, sort_key=lambda x: len(x.article_question),
                            device=device, train=True, sort=True, sort_within_batch=True)
test_iter = Iterator(test, batch_size=16, device=device, train=False, shuffle=False, sort=False)

/Users/gavin/anaconda3/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/gavin/anaconda3/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [20]:
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        options_name = "bert-base-uncased"
        self.encoder = BertForMultipleChoice.from_pretrained(options_name,num_labels=4)
    def forward(self, article_question, A,B,C,D, labels):
        enc_output = self.encoder(article_question,A.unsqueeze(0), labels=labels)
        enc_output = self.encoder(article_question,B.unsqueeze(0), labels=labels)
        enc_output = self.encoder(article_question,C.unsqueeze(0), labels=labels)
        enc_output = self.encoder(article_question,D.unsqueeze(0), labels=labels)
        loss, text_fea = enc_output[:2]
        return loss, text_fea

In [27]:
destination_folder = "Model"
def Train(model,
          optimizer,
          criterion = nn.BCELoss(),
          train_loader = train_iter,
          valid_loader = valid_iter,
          num_epochs = 10,
          eval_every = len(train_iter) // 2,
          file_path = destination_folder,
          best_valid_loss = float("Inf")):
    
    # initialize running values
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []

    # training loop
    model.train()
    for epoch in range(num_epochs):
        for ( labels,article_question, A,B,C,D ), _ in train_loader:
            labels = labels.type(torch.LongTensor)           
            labels = labels.to(device)
            article_question = article_question.type(torch.LongTensor)  
            article_question = article_question.to(device)
            A = A.type(torch.LongTensor)
            B = B.type(torch.LongTensor)
            C = C.type(torch.LongTensor)
            D = D.type(torch.LongTensor)
            output = model(article_question, A,B,C,D, labels)
            loss, _ = output

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update running values
            running_loss += loss.item()
            global_step += 1

            # evaluation step
            if global_step % eval_every == 0:
                model.eval()
                with torch.no_grad():                    
                    # validation loop
                    for ( labels, article_question, A,B,C,D), _ in valid_loader:
                        labels = labels.type(torch.LongTensor)           
                        labels = labels.to(device)
                        article_question = article_question.type(torch.LongTensor)  
                        article_question = article_question.to(device)
                        A = A.type(torch.LongTensor)
                        B = B.type(torch.LongTensor)
                        C = C.type(torch.LongTensor)
                        D = D.type(torch.LongTensor)
                        output = model(article_question, A,B,C,D, labels)
                        loss, _ = output
                        
                        valid_running_loss += loss.item()

                # evaluation
                average_train_loss = running_loss / eval_every
                average_valid_loss = valid_running_loss / len(valid_loader)
                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)

                # resetting running values
                running_loss = 0.0                
                valid_running_loss = 0.0
                model.train()

                # print progress
                print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                              average_train_loss, average_valid_loss))
                
                # checkpoint
                if best_valid_loss > average_valid_loss:
                    best_valid_loss = average_valid_loss
                    save_checkpoint(file_path + '/' + 'model.pt', model, best_valid_loss)
                    save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    
    save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    print('Finished Training!')
    
    
Train(model=BERT().to(device),optimizer = optim.Adam(model.parameters(),lr=2e-5))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

RuntimeError: shape '[-1, 128]' is invalid for input of size 16